In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import argparse

import numpy as np
import tensorflow as tf

from face_landmark import model
from face_landmark import load_dataset

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [7]:
!echo $CUDA_VISIBLE_DEVICES

1


In [143]:
dump_dir = '/barn2/yuan/datasets/300W_LP_mini/json'
data_iterator = load_dataset.DatasetIteratorWithHeatmap(dump_dir=dump_dir)
print(len(data_iterator))

18


In [136]:
dump_dir = '/barn2/yuan/datasets/300W_LP/json'
data_iterator = load_dataset.DatasetIteratorWithHeatmap(dump_dir=dump_dir)
print(len(data_iterator))

61225


In [ ]:
len(data_iterator)

In [138]:
try:
    del sess0
    del sess
except:
    pass
try:
    del graph0
    del session
except:
    pass
tf.reset_default_graph()

In [145]:
graph0 = tf.Graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(target='', graph=graph0, config=config) as sess:
    image_tensor = tf.placeholder(dtype=tf.uint8, shape=[None, None, None, 3])
    x1 = (tf.cast(x=image_tensor, dtype=tf.float32) * 2.0 / 255.0) - 1.0
    x2 = tf.image.resize_images(x1, size=(256, 256))
    heatmap_inferred_tensors = model.fan(x=x2, num_modules=1)
    heatmap_inferred_t1_tensor = heatmap_inferred_tensors[0]
    heatmap_groundtruth_tensor = tf.placeholder(dtype=tf.float32, shape=[None, 64, 64, 68])
    loss_op = tf.losses.mean_squared_error(labels=heatmap_groundtruth_tensor, predictions=heatmap_inferred_t1_tensor)
    train_op = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss_op)
    
    
    init_op = tf.global_variables_initializer()
    
    sess.run(init_op)
    
    n_epoch = 200
    n_steps = len(data_iterator)

    for i_epoch in range(n_epoch):
        loss = 0.0
        for i_step, (image_sample, heatmap_sample) in enumerate(data_iterator):
            loss_i, _ = sess.run([loss_op, train_op], 
                                  feed_dict={image_tensor:[image_sample],
                                             heatmap_groundtruth_tensor:[heatmap_sample]})
            loss += loss_i
        print('epoch:', i_epoch, 'loss:', loss / n_steps)

epoch: 0 loss: 1.4368111226293776
epoch: 1 loss: 1.1972072124481201
epoch: 2 loss: 1.0517540739642248
epoch: 3 loss: 0.9463422927591536
epoch: 4 loss: 0.8636044992340935
epoch: 5 loss: 0.7953388161129422
epoch: 6 loss: 0.7372961507903205
epoch: 7 loss: 0.6870713068379296
epoch: 8 loss: 0.6428966853353713
epoch: 9 loss: 0.6035722196102142
epoch: 10 loss: 0.5681571596198611
epoch: 11 loss: 0.5359877049922943
epoch: 12 loss: 0.5066475023825964
epoch: 13 loss: 0.47977151970068616
epoch: 14 loss: 0.4550356964270274
epoch: 15 loss: 0.4321424000793033
epoch: 16 loss: 0.4108474552631378
epoch: 17 loss: 0.39097684456242454
epoch: 18 loss: 0.3723641468418969
epoch: 19 loss: 0.35488467249605393
epoch: 20 loss: 0.3384358170959685
epoch: 21 loss: 0.32292132410738206
epoch: 22 loss: 0.3082752111885283
epoch: 23 loss: 0.29442427721288467
epoch: 24 loss: 0.281302230225669
epoch: 25 loss: 0.2688538233439128
epoch: 26 loss: 0.25703509400288266
epoch: 27 loss: 0.2457987442612648
epoch: 28 loss: 0.2351071

In [ ]:
graph0 = tf.Graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
import os

with tf.Session(target='', graph=graph0, config=config) as sess:
    record_path = '/barn2/yuan/datasets/300W_LP/flat_dataset.record'
    dataseto = load_dataset.Dataset()
    dataset = dataseto.get_tf_dataset(record_path, batch_size=16)
    dataset = dataset.prefetch(buffer_size=200)
    iterator = dataset.make_initializable_iterator()
    next_example = iterator.get_next()
    image_tensor, heatmap_groundtruth_tensor = next_example

    print(image_tensor)
    print(heatmap_groundtruth_tensor)
    
    heatmap_inferred_tensors = model.fan(x=image_tensor, num_modules=4)
    print('inferred length: ', len(heatmap_inferred_tensors))
    loss_op = 0.0
    for heatmap_inferred_tensor in heatmap_inferred_tensors:
        loss_op = loss_op + tf.losses.mean_squared_error(labels=heatmap_groundtruth_tensor, predictions=heatmap_inferred_tensor)
    train_op = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss_op)

    init_op = tf.global_variables_initializer()
    
    sess.run(init_op)
    
    os.makedirs('./saved_model/face_landmark/20180921_01', exist_ok=True)
    saver = tf.train.Saver()
    
    n_epoch = 200
    for i_epoch in range(n_epoch):
        sess.run(iterator.initializer)
        loss = 0.0
        n_steps = 0
        step_cnt = 0
        while True:
            step_cnt += 1
            try:
                loss_i, _ = sess.run([loss_op, train_op])
                loss += loss_i
                n_steps += 1
            except tf.errors.OutOfRangeError:
                saver.save(sess, './saved_model/face_landmark/20180921_01')
                break
            if step_cnt % 20 == 0:
                print('epoch:', i_epoch + 1, 'loss:', loss / n_steps)
        print('epoch:', i_epoch + 1, 'loss:', loss / n_steps)


Tensor("IteratorGetNext:0", shape=(?, 256, 256, 3), dtype=float32)
Tensor("IteratorGetNext:1", shape=(?, 64, 64, 68), dtype=float32)
in_channels: 64
out_channels: 128
in_channels: 128
out_channels: 256
previous shape: (?, 64, 64, 256)
low1 shape: (?, 32, 32, 256)
low1 shape: (?, 16, 16, 256)
low1 shape: (?, 8, 8, 256)
low1 shape: (?, 4, 4, 256)
previous shape: (?, 64, 64, 256)
low1 shape: (?, 32, 32, 256)
low1 shape: (?, 16, 16, 256)
low1 shape: (?, 8, 8, 256)
low1 shape: (?, 4, 4, 256)
previous shape: (?, 64, 64, 256)
low1 shape: (?, 32, 32, 256)
low1 shape: (?, 16, 16, 256)
low1 shape: (?, 8, 8, 256)
low1 shape: (?, 4, 4, 256)
previous shape: (?, 64, 64, 256)
low1 shape: (?, 32, 32, 256)
low1 shape: (?, 16, 16, 256)
low1 shape: (?, 8, 8, 256)
low1 shape: (?, 4, 4, 256)
inferred length:  4
epoch: 1 loss: 5.876730585098267
epoch: 1 loss: 5.567057192325592
epoch: 1 loss: 5.304811803499858
epoch: 1 loss: 5.074772834777832
epoch: 1 loss: 4.863504695892334
epoch: 1 loss: 4.671355456113815


epoch: 2 loss: 0.26487569542867795
epoch: 2 loss: 0.2646452578491178
epoch: 2 loss: 0.2642681780954202
epoch: 2 loss: 0.2638268022527618
epoch: 2 loss: 0.26303189613390715
epoch: 2 loss: 0.26258318211996196
epoch: 2 loss: 0.2621496223351535
epoch: 2 loss: 0.26173096778137345
epoch: 2 loss: 0.2615263026828567
epoch: 2 loss: 0.26114144109793613
epoch: 2 loss: 0.26029114427143024
epoch: 2 loss: 0.2602753886427635
epoch: 2 loss: 0.2595387624017894
epoch: 2 loss: 0.25932716952591406
epoch: 2 loss: 0.25834605140345435
epoch: 2 loss: 0.25750793703766756
epoch: 2 loss: 0.2569740732115778
epoch: 2 loss: 0.2562594680984815
epoch: 2 loss: 0.2559559395455796
epoch: 2 loss: 0.2555177075273179
epoch: 2 loss: 0.25496825684482854
epoch: 2 loss: 0.2546556391582197
epoch: 2 loss: 0.2539828732460737
epoch: 2 loss: 0.25357624356653174
epoch: 2 loss: 0.2530727130432542
epoch: 2 loss: 0.2526774268527076
epoch: 2 loss: 0.2522663186683699
epoch: 2 loss: 0.25178201674060385
epoch: 2 loss: 0.25148073422855566
e

In [56]:
graph = tf.get_default_graph()
print(graph.as_graph_def())

versions {
  producer: 27
}

